In [1]:
# !conda install --channel conda-forge cartopy 
# !pip install pandas keplergl movingpandas 
# !pip install topojson 
# !pip install geojson
# ! pip install reverse-geocode
# !pip install geopy 


In [73]:
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px # 3d plot


In [81]:
import geopandas as gpd
import geojson 
from tqdm.auto import tqdm 
import time
import csv


In [82]:
df_gps = pd.read_stata("raw_data/gps4_3D.dta")
df_gps.head()

,idcode,accuracy,latitude,longitude,ticon,mm_r,gg_r,MM_gps,GG_gps,hh_gps,mm_gps,ss_gps,ms_gps,time_str_gps,timestamp_gps
0,deb1cc1094abe3b84148183e5acff809c7890942,19.087999,46.06103,11.13130,network,5.0,10.0,5.0,10.0,0.0,0.0,15.0,348.0,10/5/2018 0:0:15.348,2018-05-10 00:00:15.348
1,deb1cc1094abe3b84148183e5acff809c7890942,19.167999,46.06103,11.13130,network,5.0,10.0,5.0,10.0,0.0,0.0,56.0,41.0,10/5/2018 0:0:56.41,2018-05-10 00:00:56.410
2,deb1cc1094abe3b84148183e5acff809c7890942,19.167999,46.06103,11.13130,network,5.0,10.0,5.0,10.0,0.0,0.0,56.0,95.0,10/5/2018 0:0:56.95,2018-05-10 00:00:56.950
3,deb1cc1094abe3b84148183e5acff809c7890942,19.100000,46.06104,11.13133,network,5.0,10.0,5.0,10.0,0.0,1.0,44.0,38.0,10/5/2018 0:1:44.38,2018-05-10 00:01:44.380
4,deb1cc1094abe3b84148183e5acff809c7890942,19.129999,46.06102,11.13130,network,5.0,10.0,5.0,10.0,0.0,1.0,58.0,8.0,10/5/2018 0:1:58.8,2018-05-10 00:01:58.800


In [83]:
df_gps.columns

Index(['idcode', 'accuracy', 'latitude', 'longitude', 'ticon', 'mm_r', 'gg_r',
       'MM_gps', 'GG_gps', 'hh_gps', 'mm_gps', 'ss_gps', 'ms_gps',
       'time_str_gps', 'timestamp_gps'],
      dtype='object')

In [84]:
# Drop unwanted columns
df_gps = df_gps.drop(['accuracy','ticon', 'mm_r', 'gg_r','MM_gps', 'GG_gps', 'hh_gps',\
                     'mm_gps', 'ss_gps', 'ms_gps','time_str_gps'], axis=1)
df_gps.shape

(96732, 4)

In [85]:
# check if the timestamp column  of our data is sorted / inorder to sort the gis data based on timestamp 
s = True 
for i in range(len(df_gps)-1):
    if df_gps.timestamp_gps[i] > df_gps.timestamp_gps[i+1]:
        s = False
if s : 
    print(" sorted")
else:
    print("not sorted ")

not sorted 


In [86]:
df_gps["timestamp_gps"].head()

0   2018-05-10 00:00:15.348
1   2018-05-10 00:00:56.410
2   2018-05-10 00:00:56.950
3   2018-05-10 00:01:44.380
4   2018-05-10 00:01:58.800
Name: timestamp_gps, dtype: datetime64[ns]

In [87]:
# sort the timestamp column 
df_gps= df_gps.sort_values("timestamp_gps").reset_index(drop=True)

In [88]:
df_gps["timestamp_gps"].head()

0   2018-05-10 00:00:06.865
1   2018-05-10 00:00:15.348
2   2018-05-10 00:00:35.390
3   2018-05-10 00:00:56.410
4   2018-05-10 00:00:56.950
Name: timestamp_gps, dtype: datetime64[ns]

In [10]:
df_gps.shape

(96732, 4)

### Convert the panda data frame to the geopanda  data frame  
* create a geomtry column for the long and lat 


In [11]:
gdf_gps = gpd.GeoDataFrame(df_gps,geometry=gpd.points_from_xy(df_gps.longitude,df_gps.latitude))

In [12]:
type(gdf_gps)

geopandas.geodataframe.GeoDataFrame

In [13]:
gdf_gps.shape

(96732, 5)

In [14]:
gdf_gps.head()

,idcode,latitude,longitude,timestamp_gps,geometry
0,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:06.865,POINT (11.13661 46.06229)
1,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:15.348,POINT (11.13130 46.06103)
2,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:35.390,POINT (11.13661 46.06229)
3,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.410,POINT (11.13130 46.06103)
4,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.950,POINT (11.13130 46.06103)


### Calculate x and y coordinates

* For coordinates captured using a GPS, or by any means, longitude is the X value and latitude is the Y value.
* These are for a geographic coordinate system and have units of degrees.[Here](https://gis.stackexchange.com/questions/260672/difference-between-x-y-and-latitude-longitude-or-meters#:~:text=For%20coordinates%20captured%20using%20a,and%20have%20units%20of%20degrees.)
*let'scalculate x and y coordinates based on our Point geometries, so that we can easily plot our data in 3D:


In [15]:
gdf_gps["x"] = gdf_gps["geometry"].x  # x  is longitude 
gdf_gps["y"] = gdf_gps["geometry"].y   # y is latitude 


In [16]:
gdf_gps.shape

(96732, 7)

In [17]:
gdf_gps.head()

,idcode,latitude,longitude,timestamp_gps,geometry,x,y
0,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:06.865,POINT (11.13661 46.06229),11.13661,46.06229
1,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:15.348,POINT (11.13130 46.06103),11.13130,46.06103
2,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:35.390,POINT (11.13661 46.06229),11.13661,46.06229
3,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.410,POINT (11.13130 46.06103),11.13130,46.06103
4,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.950,POINT (11.13130 46.06103),11.13130,46.06103


# location set

In [18]:
locations = set()
for index,row in gdf_gps.iterrows():
    locations.add((row.x,row.y))

In [19]:
len(locations)

9123

In [20]:
locations = list(locations)
locations[:10]

[(11.13118, 46.06076),
 (11.11856, 46.06598),
 (11.1309, 46.0612),
 (16.41321, 41.15685),
 (11.13103, 46.06125),
 (11.11804, 46.06604),
 (11.13865, 46.06504),
 (11.11886, 46.0699),
 (16.41363, 41.15684),
 (11.11864, 46.06602)]

# location to name

In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Trento")
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [22]:
# location_to_name = dict()
# for lon,lat in tqdm(locations[0:2000],total = len(locations[0:2000])):
#     try:
#         location = geolocator.reverse("{},{}".format( lat, lon),timeout=None)
#     except:
#         time.sleep(1)
#         location = geolocator.reverse("{},{}".format(lat, lon),timeout=None)

#     address = location.raw["address"]
#     if "road" in address and "suburb" in address and "city" in address:
#         result = "{}-{}-{}".format(address["city"],address["suburb"],address["road"])
#     elif "suburb" in address and "city" in address:
#         results = "{}-{}".format(address["city"],address["suburb"])
#     elif "city" in address:
#         result = "{}".format(address["city"])
#     else:
#         result = ""
#     location_to_name[(lon,lat)] = result

In [23]:
# for lon,lat in tqdm(locations[2000:4000],total = len(locations[2000:4000])):
#     try:
#         location = geolocator.reverse("{},{}".format( lat, lon),timeout=None)
#     except:
#         time.sleep(1)
#         location = geolocator.reverse("{},{}".format(lat, lon),timeout=None)

#     address = location.raw["address"]
#     if "road" in address and "suburb" in address and "city" in address:
#         result = "{}-{}-{}".format(address["city"],address["suburb"],address["road"])
#     elif "suburb" in address and "city" in address:
#         results = "{}-{}".format(address["city"],address["suburb"])
#     elif "city" in address:
#         result = "{}".format(address["city"])
#     else:
#         result = ""
#     location_to_name[(lon,lat)] = result

In [24]:
# for lon,lat in tqdm(locations[4000:6000],total = len(locations[4000:6000])):
#     try:
#         location = geolocator.reverse("{},{}".format( lat, lon),timeout=None)
#     except:
#         time.sleep(1)
#         location = geolocator.reverse("{},{}".format(lat, lon),timeout=None)

#     address = location.raw["address"]
#     if "road" in address and "suburb" in address and "city" in address:
#         result = "{}-{}-{}".format(address["city"],address["suburb"],address["road"])
#     elif "suburb" in address and "city" in address:
#         results = "{}-{}".format(address["city"],address["suburb"])
#     elif "city" in address:
#         result = "{}".format(address["city"])
#     else:
#         result = ""
#     location_to_name[(lon,lat)] = result

In [25]:
# for lon,lat in tqdm(locations[6000:8000],total = len(locations[6000:8000])):
#     try:
#         location = geolocator.reverse("{},{}".format( lat, lon),timeout=None)
#     except:
#         time.sleep(1)
#         location = geolocator.reverse("{},{}".format(lat, lon),timeout=None)

#     address = location.raw["address"]
#     if "road" in address and "suburb" in address and "city" in address:
#         result = "{}-{}-{}".format(address["city"],address["suburb"],address["road"])
#     elif "suburb" in address and "city" in address:
#         results = "{}-{}".format(address["city"],address["suburb"])
#     elif "city" in address:
#         result = "{}".format(address["city"])
#     else:
#         result = ""
#     location_to_name[(lon,lat)] = result

In [26]:
# for lon,lat in tqdm(locations[8000:],total = len(locations[8000:])):
#     try:
#         location = geolocator.reverse("{},{}".format( lat, lon),timeout=None)
#     except:
#         time.sleep(1)
#         location = geolocator.reverse("{},{}".format(lat, lon),timeout=None)

#     address = location.raw["address"]
#     if "road" in address and "suburb" in address and "city" in address:
#         result = "{}-{}-{}".format(address["city"],address["suburb"],address["road"])
#     elif "suburb" in address and "city" in address:
#         results = "{}-{}".format(address["city"],address["suburb"])
#     elif "city" in address:
#         result = "{}".format(address["city"])
#     else:
#         result = ""
#     location_to_name[(lon,lat)] = result

In [27]:
# len(location_to_name.values())

In [28]:
gdf_gps.shape

(96732, 7)

In [29]:
# set(location_to_name.values())

In [30]:
# with open('location.csv', 'w') as csv_file:  
#     writer = csv.writer(csv_file)
#     for key, value in location_to_name.items():
#         writer.writerow([key, value])

In [31]:
gdf_gps.head()

,idcode,latitude,longitude,timestamp_gps,geometry,x,y
0,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:06.865,POINT (11.13661 46.06229),11.13661,46.06229
1,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:15.348,POINT (11.13130 46.06103),11.13130,46.06103
2,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:35.390,POINT (11.13661 46.06229),11.13661,46.06229
3,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.410,POINT (11.13130 46.06103),11.13130,46.06103
4,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.950,POINT (11.13130 46.06103),11.13130,46.06103


In [32]:
df_loc = pd.read_csv("location.csv", header=None, names=['lat_lon', 'Adress'])

In [33]:
df_loc.shape

(9123, 2)

In [34]:
df_loc.head()

,lat_lon,Adress
0,"(11.13118, 46.06076)",Trento-Bolghera-Via Adamello
1,"(11.11856, 46.06598)",Trento-Le Albere-Via Vigilio Inama
2,"(11.1309, 46.0612)",Trento-Bolghera-Viale Nepomuceno Bolognini
3,"(16.41321, 41.15685)",NaN
4,"(11.13103, 46.06125)",Trento-Bolghera-Viale Nepomuceno Bolognini


In [35]:
df_loc.isna().sum()

lat_lon       0
Adress     1116
dtype: int64

In [36]:
#python pandas dataframe columns convert to dict key and value
loc_dict  = dict(zip(df_loc.lat_lon, df_loc.Adress))


In [37]:
len(loc_dict)

9123

In [38]:
# # append location name as a column on the data frame 
gdf_gps["location_name"] = gdf_gps.apply(lambda row:loc_dict[str((row.x,row.y))],axis=1)

In [39]:
gdf_gps.shape

(96732, 8)

In [40]:
gdf_gps.head()

,idcode,latitude,longitude,timestamp_gps,geometry,x,y,location_name
0,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:06.865,POINT (11.13661 46.06229),11.13661,46.06229,Trento-Bolghera-Via Montello
1,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:15.348,POINT (11.13130 46.06103),11.13130,46.06103,Trento-Bolghera-Viale Nepomuceno Bolognini
2,f9b89058526285256897833d8e21eab988b6057f,46.06229,11.13661,2018-05-10 00:00:35.390,POINT (11.13661 46.06229),11.13661,46.06229,Trento-Bolghera-Via Montello
3,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.410,POINT (11.13130 46.06103),11.13130,46.06103,Trento-Bolghera-Viale Nepomuceno Bolognini
4,deb1cc1094abe3b84148183e5acff809c7890942,46.06103,11.13130,2018-05-10 00:00:56.950,POINT (11.13130 46.06103),11.13130,46.06103,Trento-Bolghera-Viale Nepomuceno Bolognini


In [41]:
gdf_gps["timestamp_gps"].min()

Timestamp('2018-05-10 00:00:06.865000')

* The data started being collected on 10th of may 2018  

In [42]:
gdf_gps["timestamp_gps"].max()

Timestamp('2018-05-23 22:26:49.319000')

* The data collection endded on 23 th of may 2018  (after 13 days)

In [43]:
# we have 13 days of data 
gdf_gps["timestamp_gps"].max()-gdf_gps["timestamp_gps"].min()

Timedelta('13 days 22:26:42.454000')

In [44]:
# number of partcipant 
gdf_gps["idcode"].unique().tolist()


['f9b89058526285256897833d8e21eab988b6057f',
 'deb1cc1094abe3b84148183e5acff809c7890942',
 'f88da12a9900972539558ad26ee4d65b48eef06f']

* We have 3 partcipants / 3 subjects for this study 

In [45]:
sub_1,sub_2,sub_3= gdf_gps["idcode"].unique().tolist()


In [46]:
gdf_sub1 = gdf_gps[gdf_gps["idcode"]==sub_1].reset_index(drop=True)
gdf_sub2 = gdf_gps[gdf_gps["idcode"]==sub_2].reset_index(drop=True)
gdf_sub3 = gdf_gps[gdf_gps["idcode"]==sub_3].reset_index(drop=True)
print(gdf_sub1.shape)
print(gdf_sub2.shape)
print(gdf_sub3.shape)

(19570, 8)
(47054, 8)
(30108, 8)


In [47]:
gdf_sub1["timestamp_gps"].max()-gdf_sub1["timestamp_gps"].min()

Timedelta('13 days 20:40:59.235000')

In [48]:
gdf_sub2["timestamp_gps"].max()-gdf_sub2["timestamp_gps"].min()

Timedelta('13 days 20:06:56.086000')

In [49]:
gdf_sub3["timestamp_gps"].max()-gdf_sub3["timestamp_gps"].min()

Timedelta('13 days 22:24:23.510000')

* We have 13 days of data for all of our  partcipants / subjects only the time diffrence found


In [50]:
print(gdf_sub1.shape)
print(gdf_sub2.shape)
print(gdf_sub3.shape)

(19570, 8)
(47054, 8)
(30108, 8)


In [51]:
df_a = pd.DataFrame(gdf_gps.idcode.value_counts()).reset_index(drop=False)
df_a.columns = ["idcode","count"]
# rename idcode value
df_a["idcode"].replace({df_a.idcode[0]: "Subject_1",df_a.idcode[1]: "Subject_2",df_a.idcode[2]: "Subject_3"}, inplace=True)



In [56]:

fig = px.bar(df_a, x="idcode", y="count", color="idcode", title="Valu count of data for each subject")



fig.show()

* value counts of our partcipant, according to the chart subject one has more data

## Export the GIS data of each subject as geogeson file

In [53]:
# # let's save our geopanda of each subject 
# gdf_sub1['timestamp_gps'] = gdf_sub1['timestamp_gps'].dt.strftime('%Y-%m-%d %H:%M:%S')

# with open("gdata/gdf_sub1.geojson", 'w') as f:
#     geojson.dump(gdf_sub1, f,)

In [54]:
# # let's save our geopanda of each subject 
# gdf_sub2['timestamp_gps'] = gdf_sub2['timestamp_gps'].dt.strftime('%Y-%m-%d %H:%M:%S')

# with open("gdata/gdf_sub2.geojson", 'w') as f:
#     geojson.dump(gdf_sub2, f,)

In [55]:
# # let's save our geopanda of each subject 
# gdf_sub3['timestamp_gps'] = gdf_sub3['timestamp_gps'].dt.strftime('%Y-%m-%d %H:%M:%S')

# with open("gdata/gdf_sub3.geojson", 'w') as f:
#     geojson.dump(gdf_sub3, f,)